# Premières requêtes sur API SIRENE

**Ceci est un notebook Jupyter Python** (l'extension .ipynb du fichier est caractéristique).   
Cela signifie qu'il peut se lire joliment sous github mais aussi se charger dans Jupyter pour exécution (modification, trifouillage, au choix).   
Il suffit de le télécharger (ou de cloner tout le repository pour ceux qui sont un peu git-compatible) et de mettre dans votre instance Jupyter (si vous êtes vraiment de mauvaise humeur, vous pouvez faire afficher 'raw' dans github pour avoir le code source de la page et créer un nouveau notebook en copiant/collant celui-ci dans un fichier vierge).

## Utilisation de la librairie requests
[La documentation officielle](http://docs.python-requests.org/en/v0.10.6/api/) en anglais, [une version en français](http://fr.python-requests.org/en/latest/user/quickstart.html) et un [tutoriel](http://dridk.me/python-requests.html) également en français. 

In [2]:
import requests
import pandas

## Une première requête

Construire un requête est simple :
* on définit l'URL de la requête (sous forme de chaîne de caractère),

In [3]:
url = "https://prototype.api-sirene.insee.fr/ws/siret/"

* si besoin est, on définit les paramètres de la requête (sous forme de dictionnaire),

In [4]:
##On a choisit pour l'exemple de s'en prendre aux notaires (oui c'est gratuit, non je n'ai pas envie d'en parler...).
q_param_value = 'Denomination:Notaire'
requete_params = {'q':q_param_value}

* si besoin est, on définit les urls des proxies (sous forme de dictionnaire)

In [5]:
insee_proxies = {'http': 'http://proxy-rie.http.insee.fr:8080',
              'https': 'http://proxy-rie.http.insee.fr:8080'}

* on passe le tout en paramètre de la fonction correspondant au verbe HTTP souhaité (get, post, put, delete... et selon le contexte avec ou sans le param proxies)

In [7]:
#Requête dans un contexte sans proxy
#requete = requests.get(url, params=requete_params)
#Requête avec proxy
requete = requests.get(url, params=requete_params, proxies=insee_proxies)

On peut vérifier que la requête s'est bien passée soit :
* en vérifiant la valeur du code retour qui doit valoir 200

In [8]:
assert requete.status_code == 200, "Erreur lors de la requête"

* ou en utilisant une fonction ad-hoc

In [9]:
requete.raise_for_status()
##Aucun retour si la requête a renvoyé 200

## Accéder au contenu de la requête

API SIRENE renvoie du JSON (oui, il y a tout le JSON, mais ainsi vous l'aurez déjà vu une fois).

In [10]:
requete.text

'{"Header":{"Statut":200,"Message":"ok","Total":831,"Debut":0,"Nombre":20},"Etablissements":[{"Siren":"792798415","Nic":"00016","StatutDiffusionEtablissement":"O","DateCreationEtablissement":"2013-04-01","NombrePeriodes":1,"UniteLegale":{"StatutDiffusionUniteLegale":"O","DateCreationUniteLegale":"2013-04-01","Sigle":null,"Sexe":null,"NomUsage":null,"Prenom1":null,"Prenom2":null,"Prenom3":null,"Prenom4":null,"PrenomUsuel":null,"CaractereEmployeur":"N","EtatAdministratif":"A","Nom":null,"Denomination":"NOTAIRE","ActivitePrincipale":"01.11Z","LibelleActivite":"Culture de céréales (à l\'exception du riz), de légumineuses et de graines oléagineuses","NomenclatureActivite":"NAFRev2","CategorieJuridique":"6597","CategorieProfessionnelle":null,"NICSiege":"00016"},"Adresse":{"ComplementAdresseEtablissement":null,"NumeroVoieEtablissement":"2","IndiceRepetitionEtablissement":null,"TypeVoieEtablissement":"CHE","LibelleVoieEtablissement":"DE L ORME","CodePostalEtablissement":"51350","LibelleCommune

Pour pouvoir manipuler les données, il faut d'abord les 'parser'...   
Pour Python il s'agit encore du résultat d'une requête HTTP.

In [11]:
requete.keys()

AttributeError: 'Response' object has no attribute 'keys'

Parser le contenu de la réponse est triviale (quand le contenu/body respecte un format supporté)

In [12]:
data_raw = requete.json()

Maintenant on dispose d'un dictionnaire Python classique (cf. [les dictionnaires Python](https://www.tutorialspoint.com/python/python_dictionary.htm) au besoin)

In [13]:
data_raw.keys()

dict_keys(['Header', 'Etablissements'])

Le contenu qui nous intéresse correspond à la clé Etablissements, qui elle contient une liste des établissements correspondant à la requête.

In [14]:
data = data_raw["Etablissements"]
type(data)

list

## Autres requêtes

On peut aussi requêter l'API pour avoir l'historique complet (entreprise existante et fermée) à partir de la racine 'siren'.

In [15]:
url2 = "https://prototype.api-sirene.insee.fr/ws/siren/"
bad_requete2 = requests.get(url2 , params=requete_params)
bad_requete2.raise_for_status()

ConnectionError: HTTPSConnectionPool(host='prototype.api-sirene.insee.fr', port=443): Max retries exceeded with url: /ws/siren/?q=Denomination%3ANotaire (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4d0eedd0f0>: Failed to establish a new connection: [Errno -2] Name or service not known',))

``Attention`` les paramètres attendus non pas tout à fait la même syntaxe...

In [18]:
q2_param_value = 'periode(Denomination:Notaire)'
requete2_params = {'q':q2_param_value}
#requete2 = requests.get(url2 , params=requete2_params)
requete2 = requests.get(url2 , proxies=insee_proxies,params=requete2_params)
requete2.raise_for_status()

Cette fois une clé Periodes renseigne sur les différents états des unités légales renvoyées.

Si on est joueur, il est également possible de requêter avec des critères plus complexes :   
* Plusieurs termes pour une même variable :

In [30]:
q3_param_value = 'Denomination:Notaire AND Denomination:Bénévole'
requete3_params = {'q':q3_param_value}
requete3 = requests.get(url, params=requete3_params, proxies=insee_proxies)
requete3.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: https://prototype.api-sirene.insee.fr/ws/siret/?q=Denomination%3ANotaire+AND+Denomination%3AB%C3%A9n%C3%A9vole

Erreur 404 ?? Mais pourquoi donc... Ah ben si, tout simplement, il n'existe pas de notaires bénévoles.  
A retenir, on obtiendra une erreur 404 (signifiant "Not Found") quand notre requête ne renvoie aucun résultat, donc pas de panique si vous obtenez une 404, il n'y a juste aucun écho à votre requête.

Cherchons quelque chose de plus réaliste (et qui peut aussi servir, sait-on jamais...) :

In [35]:
q4_param_value = 'Denomination:Notaire AND LibelleCommuneEtablissement:Marseille'
requete4_params = {'q':q4_param_value}
requete4 = requests.get(url, params=requete4_params, proxies=insee_proxies)
requete4.raise_for_status()

In [36]:
requete4.text

'{"Header":{"Statut":200,"Message":"ok","Total":7,"Debut":0,"Nombre":7},"Etablissements":[{"Siren":"394675730","Nic":"00011","StatutDiffusionEtablissement":"O","DateCreationEtablissement":"1994-03-10","NombrePeriodes":4,"UniteLegale":{"StatutDiffusionUniteLegale":"O","DateCreationUniteLegale":"1994-03-10","Sigle":null,"Sexe":null,"NomUsage":null,"Prenom1":null,"Prenom2":null,"Prenom3":null,"Prenom4":null,"PrenomUsuel":null,"CaractereEmployeur":"O","EtatAdministratif":"A","Nom":null,"Denomination":"ALEXANDRE LAFAGE NOTAIRE ASSOCIE","ActivitePrincipale":"69.10Z","LibelleActivite":"Activités juridiques","NomenclatureActivite":"NAFRev2","CategorieJuridique":"6565","CategorieProfessionnelle":null,"NICSiege":"00011"},"Adresse":{"ComplementAdresseEtablissement":null,"NumeroVoieEtablissement":"20","IndiceRepetitionEtablissement":null,"TypeVoieEtablissement":"BD","LibelleVoieEtablissement":"GAROUTTE","CodePostalEtablissement":"13012","LibelleCommuneEtablissement":"MARSEILLE","LibelleCommuneEtra

* Plusieurs critères dont un en exclusion :

A l'inverse, des afficionados du PSG peuvent être prêts à tout acheter, sauf à Marseille... Avec API SIRENE, c'est possible !

In [37]:
q5_param_value = 'Denomination:Notaire AND -LibelleCommuneEtablissement:Marseille'
requete5_params = {'q':q4_param_value}
requete5 = requests.get(url, params=requete5_params, proxies=insee_proxies)
requete5.raise_for_status()

A noter le signe '-' devant le nom de variable.  
Il est possible d'utiliser également l'opérateur 'OR' de la même manière, attention aux règles de préséance des opérateurs, ne pas hésiter à user de parenthèses pour lever toute ambiguëté.

* Un raccourci pour la route :  
    Il est possible d'utiliser le signe '-' pour l'opérateur 'OR' portant sur deux termes de recherche pour une même variable.

In [38]:
q5_param_value = 'Denomination:Notaire-Bénévole'
requete5_params = {'q':q4_param_value}
requete5 = requests.get(url, params=requete5_params, proxies=insee_proxies)
requete5.raise_for_status()

Bon là on a pas d'erreur 404, puisqu'on récupère les notaires et les bénévoles (deux ensembles que l'on sait parfaitement disjoints)